In [64]:
# Necessary library imports
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time

%matplotlib inline

In [65]:
# Importing the data from file
parking_counts = pd.read_csv('data/Parking_Counts.csv')
restaurant_counts = pd.read_csv('data/Restaurant_Counts.csv')
stop_counts = pd.read_csv('data/Stop_Counts.csv')

In [75]:
# Fixing and cleaning the data to suit purpose
restaurant_counts.rename(columns={'lat':'latitude'}, inplace=True)
restaurant_counts.rename(columns={'lon':'longitude'}, inplace=True)
restaurant_counts.rename(columns={'AM early':'am_early'}, inplace=True)
restaurant_counts.rename(columns={'AM peak':'am_peak'}, inplace=True)
restaurant_counts.rename(columns={'Mid-day':'midday'}, inplace=True)
restaurant_counts.rename(columns={'PM peak':'pm_peak'}, inplace=True)
restaurant_counts.rename(columns={'PM late':'pm_late'}, inplace=True)
restaurant_counts.rename(columns={'Total Score':'total_count'}, inplace=True)
restaurant_counts.rename(columns={'Type of Place':'type'}, inplace=True)
restaurant_counts.rename(columns={'Unnamed: 0':'id'}, inplace=True)
parking_counts.rename(columns={'pole_id':'id'}, inplace=True)
stop_counts.rename(columns={'stop_id':'id'}, inplace=True)
parking_counts['type'] = 'parking'
stop_counts['type'] = 'transit'
location_counts = restaurant_counts.append(parking_counts)
# Setting a new index
#location_counts.index = idx # new ad hoc index
location_counts.index = range(len(location_counts)) # set with list
location_counts = location_counts.reset_index() # replace old w new
location_counts.rename(columns={'index':'idx'}, inplace= True)
location_counts.rename(columns={'id':'type_id'}, inplace= True)
del location_counts['Unnamed: 0']

location_counts['row'] = 0
location_counts['col'] = 0
#location_counts['longitude']

0      -117.096294
1      -117.096250
2      -117.126968
3      -117.096250
4      -117.157817
5      -117.107167
6      -117.150878
7      -117.156144
8      -117.075608
9      -117.160357
10     -117.991337
11     -116.933995
12     -117.119582
13     -117.346091
14     -117.167770
15     -117.160403
16     -117.216603
17     -117.027080
18     -117.162257
19     -117.250570
20     -117.209735
21     -117.196907
22     -117.221017
23     -117.127964
24     -117.078602
25     -117.233095
26     -117.211195
27     -117.248476
28     -117.139860
29     -117.250993
           ...    
7773   -117.162015
7774   -117.161984
7775   -117.161963
7776   -117.161932
7777   -117.161904
7778   -117.161870
7779   -117.161842
7780   -117.161818
7781   -117.161780
7782   -117.161756
7783   -117.161725
7784   -117.161705
7785   -117.161666
7786   -117.161645
7787   -117.161614
7788   -117.161389
7789   -117.161069
7790   -117.160891
7791   -117.160796
7792   -117.160702
7793   -117.160591
7794   -117.

## Initializing the grid

In [67]:
# Getting the values of the grid
long_max = location_counts['longitude'].max()
long_min = location_counts['longitude'].min()
lat_max = location_counts['latitude'].max()
lat_min = location_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min

NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
def set_gridspace(split_num):
    lat_gap = lat_dif / split_num
    long_gap = long_dif / split_num
set_gridspace(NUMBER_BLOCKS_ROOT)
long_min, long_max, lat_dif, long_dif, lat_gap, long_gap

(-180.0,
 -116.7536147,
 2.1803627000000034,
 63.2463853,
 0.21803627000000034,
 6.3246385299999996)

## Algorithm 1

Issue with the labeling of rows and cols, I believe its a float calculation problem

In [69]:
# Creating a classifier function which runs through values in dataset to get values
def classify_blocks(s):
    location_counts.loc[location_counts['idx'] == s,'row'] =  (location_counts.loc[location_counts['idx'] == s,'latitude'] - lat_min) // lat_gap
    location_counts.loc[location_counts['idx'] == s,'col'] =  (location_counts.loc[location_counts['idx'] == s,'longitude'] - long_min) // long_gap

location_counts['idx'].apply(classify_blocks)

location_counts.loc[location_counts['row'] == -0.0, 'row'] = 10.0
location_counts.loc[location_counts['col'] == -0.0, 'col'] = 10.0
#location_counts.to_csv('Label_issue.csv')
location_counts

10.0

## Algorithm 2

Issues with iterating through the dataframe to get and set values

In [27]:
# Creating the grid with row, col and lat, lon pair
grid = []
for i in range(NUMBER_BLOCKS_ROOT):
    for j in range(NUMBER_BLOCKS_ROOT):
        grid.append([i, j, (lat_min+ i*lat_gap), (long_min + j*long_gap)])    
grid = pd.DataFrame(grid)
grid.columns = ['row','col','lat','lon']
grid

,row,col,lat,lon
0,0,0,32.100000,-180.000000
1,0,1,32.100000,-173.675361
2,0,2,32.100000,-167.350723
3,0,3,32.100000,-161.026084
4,0,4,32.100000,-154.701446
5,0,5,32.100000,-148.376807
6,0,6,32.100000,-142.052169
7,0,7,32.100000,-135.727530
8,0,8,32.100000,-129.402892
9,0,9,32.100000,-123.078253


In [43]:
# Initializing dataframes for getting values for latitudes and longitudes in the dataframe
grid_lat = grid[['lat']].drop_duplicates().reset_index().sort_index(ascending=True)
del grid_lat['index']
grid_lon = grid[['lon']].drop_duplicates().reset_index().sort_index(ascending=True)
del grid_lon['index']
#grid_lat
grid_lon

,lon
0,-180.000000
1,-173.675361
2,-167.350723
3,-161.026084
4,-154.701446
5,-148.376807
6,-142.052169
7,-135.727530
8,-129.402892
9,-123.078253


# Implementation 1 for get_row using pandas dataframe

In [51]:
#Implentation #1 using pandas dataframe
def get_row(ilat):
    row_i = grid_lat[grid_lat['lat'] > ilat].idxmin()
    #row_i = np.where(grid_lat>ilat)[0].min()
    #row_i = np.where((grid_lat['lat'] > ilat))[0][0]
    return row_i

def get_col(ilon):
    #col_i = np.where((grid_lon['lon'] > ilon))[0][0]
    col_i = grid_lon[grid_lon['lon'] > ilon].idxmin()
    #col_i = np.where(grid_cols>ilon)[0].min()
    return col_i

#testing out function
# row 7(lat 8) min value = 33.626254
print get_row(33.626255)
#testing out function get_col
# col 7(lon 8) min value = -135.727530
print get_col(-135.727529)

lat    8
dtype: int64
lon    8
dtype: int64


# Implementation 2 for get_row using np.where() and np.min()

In [53]:
#Alternate implementation using np.where() and min()
def get_row_2(ilat):
    #row_i = grid_lat[grid_lat['lat'] > ilat].idxmin()
    row_i = np.where(grid_lat>ilat)[0].min()
    #row_i = np.where((grid_lat['lat'] > ilat))[0][0]
    return row_i

def get_col_2(ilon):
    #col_i = np.where((grid_lon['lon'] > ilon))[0][0]
    #col_i = grid_lon[grid_lon['lon'] > ilon].idxmin()
    col_i = np.where(grid_lon>ilon)[0].min()
    return col_i

#testing out function get_row
# row 7 min value = 33.626254
print get_row_2(33.626255)
#testing out function get_col
# col 7(lon 8) min value = -135.727530
print get_col_2(-135.727529)

8
8


## Having issues here mainly, iterating through the dataframe to set values

In [62]:
#Dunno how to iterate over dataframe without indexing error
for i in range(len(location_counts)):
    lat_i = location_counts.loc[i, 'latitude']
    lon_i = location_counts.loc[i, 'longitude']
    
    # Gets all the values that satisfy
    #print lat_i
    #print lon_i
    
    # This prints out all the correct rows till the end where i get indexing error
    #print get_row(lat_i)
    print get_row_2(lat_i)
    
    # Gets values of row and col
    #row_i = get_row(lat_i)
    #row_i = get_row_2(lat_i)
    
    #col_i = get_col(lat_i)
    #col_i = get_col_2(lat_i)
    
    # Sets the row values based on row_i
    #location_counts.set_value(i, 'row', row_i)
    #location_counts.loc[i,'row'] = row_i
    
    # Sets the col values based on col_i
    #location_counts.set_value(i, 'row', row_i)
    #location_counts.loc[i,'row'] = row_i
    
    
    #FAILED ATTEMPTS
    #row_i = row_i.index[0]
    #col_i = grid_lon[grid_lon['lon'] > lon_i]
    #col_i = col_i.iloc[[0]]
    #col_i = col_i.index
    #print row_i[0], lat_i
    
    #location_counts.loc[i,'col'] = get_col(lon_i)
    #print row_i['index'][0]
    #print grid_lon['lon'][0]
    
    #col_i = grid_lon[(grid_lon['lon'] > lon_i)].reset_index()
    #location_counts.loc[i,'row'] = row_i['index'][0]
    #location_counts.loc[i,'col'] = col_i['index'][0]
#location_counts

4
3
3
3
3
3
4
4
5
3
8
4
4
9
3
3
4
3
3
3
4
3
4
4
5
4
3
4
3
6
3
3
4
3
4
3
3
3
3
3
4
5
3
4
4
3
3
3
3
4
3
4
5
3
3
3
4
3
4
3
3
3
5
4
3
4
4
3
4
3
4
4
4
3
3
4
4
4
5
4
3
4
3
3
4
4
3
3
4
4
4
3
4
4
3
5
4
4
8
3
4
4
3
3
3
3
3
4
4
3
4
3
4
3
4
3
3
5
3
4
4
3
4
3
4
4
3
4
5
4
4
3
3
3
3
3
3
3
3
3
3
3
4
4
5
3
3
3
4
4
3
3
4
3
3
4
3
5
3
4
3
3
3
3
4
4
4
3
4
4
4
4
3
3
3
4
4
4
4
3
3
4
3
5
7
4
3
4
4
3
4
4
3
4
4
3
4
4
5
3
3
3
3
3
4
3
3
3
3
4
3
4
3
4
4
3
3
3
4
4
4
5
3
4
4
3
3
4
3
3
3
3
3
4
4
4
4
4
3
3
4
4
3
4
4
3
4
3
3
3
4
4
4
3
4
4
4
4
4
4
4
5
3
3
4
3
4
5
4
4
5
4
3
4
4
3
4
3
3
4
3
4
5
3
5
4
4
5
4
3
5
4
4
3
3
4
3
4
4
4
4
3
4
3
3
3
4
4
3
3
4
4
4
4
4
4
3
3
3
4
3
3
3
3
3
3
4
5
3
4
4
3
4
5
4
4
4
4
3
3
4
4
3
3
3
3
4
3
4
3
5
4
4
4
4
4
4
4
3
4
3
3
3
4
4
3
4
4
4
3
4
3
4
4
4
4
3
4
3
4
4
4
3
3
3
3
3
3
4
3
4
3
4
4
4
3
3
4
3
3
3
4
3
4
3
3
4
3
3
4
3
3
4
3
3
5
3
3
4
4
3
4
4
3
3
4
3
3
4
3
4
4
4
4
4
3
4
4
3
4
3
5
4
3
4
5
3
3
5
4
3
4
4
3
4
3
3
4
4
4
4
5


ValueError: zero-size array to reduction operation minimum which has no identity

In [61]:
def classify_blocks(s):
    lat_i = location_counts.loc[location_counts['idx'] == s,'latitude']
    lon_i = location_counts.loc[location_counts['idx'] == s,'longitude']
    #col_i = grid_cols[grid_cols['lon'] > lon_i].idxmin()
    #print row_i, col_i
    location_counts.loc[location_counts['idx'] == s,'row'] = get_row(lat_i)
    location_counts.loc[location_counts['idx'] == s,'col'] = get_col(lon_i)

#location_counts['idx'].apply(classify_blocks)
#location_counts

Continuing with the analysis for getting the next part which is finding outliers for grid location.

In [63]:
#Initializing random grid values
test = location_counts[['total_count','row','col']]
test['row'] = np.random.randint(0,high=10,size=len(test))
#test['row']

test['col'] = np.random.randint(0,high=10,size=len(test))
test

/Users/raghav/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/raghav/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,total_count,row,col
0,70.000000,2,2
1,70.000000,7,2
2,119.000000,7,3
3,70.000000,4,5
4,70.000000,5,6
5,119.000000,3,0
6,70.000000,1,8
7,119.000000,6,2
8,93.000000,6,5
9,119.000000,4,2


In [16]:
test_totals = test.groupby(['row','col']).sum()
test_counts = test.groupby([])
test

total_count
row col             
0   0    3130.388310
    1    4451.917887
    2    2964.799225
    3    2117.089296
    4    3648.892183
    5    3335.824507
    6    3370.822746
    7    2722.143451
    8    3593.076056
    9    3278.868944
1   0    3780.223592
    1    3214.303944
    2    2694.374085
    3    4530.233028
    4    3174.467113
    5    3724.883873
    6    3751.218310
    7    2623.777113
    8    2922.952042
    9    3928.564366
2   0    3872.555211
    1    3939.410423
    2    3539.439225
    3    3940.677465
    4    3094.760352
    5    3682.381408
    6    3004.670141
    7    3384.347606
    8    3171.416901
    9    3376.702958
...              ...
7   0    2800.321690
    1    3392.379507
    2    3381.471972
    3    3283.164648
    4    3219.087113
    5    3433.147042
    6    3025.835282
    7    3424.321479
    8    3125.536479
    9    3645.976197
8   0    3512.488028
    1    2283.127394
    2    3797.716127
    3    3244.052394
    4    2959.196620
    5    2980.855775
    6    2598.259296
    7    3112.104577
    8    3246.762817
    9    2777.175845
9   0    2933.063662
    1    3454.882042
    2    3511.211408
    3    2445.092817
    4    4087.397394
    5    3922.507817
    6    2507.162394
    7    3277.037817
    8    3578.627746
    9    2618.551056

[100 rows x 1 columns]

In [17]:
#Computes the mean for each grid position and gives the grid which seem to have outliers.
mean = test['total_count'].mean()
#print test['total_count']
std = np.std(test['total_count'])
#print std
std_test = test[(test['total_count'] >= (mean+2*std)) | (test['total_count'] <= (mean-2*std))]
std_test

total_count
row col             
0   1    4451.917887
    3    2117.089296
1   3    4530.233028
5   7    4485.772746
6   0    4967.498028